In [ ]:
### Perform the ADF test to check for stationarity

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from tqdm import tqdm  # Import tqdm for progress tracking

# Assuming your time series data is in a DataFrame called 'merged_df' with a column 'date' and multiple time series columns

# Define the significance level for the ADF test
alpha = 0.05

# Create a progress bar using tqdm
pbar = tqdm(merged_df.columns[1:], desc="Performing ADF Test", ncols=100)

# Create a dictionary to store ADF test results
adf_results = {}

# Iterate through each time series column (excluding 'date')
for col in pbar:
    # Check if the column contains non-numeric values
    if pd.to_numeric(merged_df[col], errors='coerce').notna().all():
        # Check for missing values (NaN or infinite)
        if merged_df[col].isnull().any() or not np.isfinite(merged_df[col]).all():
            print(f"Skipping '{col}' due to missing or infinite values")
            adf_results[col] = "Skipped"
        else:
            # Perform Augmented Dickey-Fuller Test for stationarity
            adf_result = adfuller(merged_df[col])
            p_value = adf_result[1]

            # Check stationarity based on the p-value
            if p_value <= alpha:
                adf_results[col] = f"Stationary (p-value: {p_value:.4f})"
            else:
                adf_results[col] = f"Non-Stationary (p-value: {p_value:.4f})"
            pbar.set_description(f"ADF Test Result for '{col}': {adf_results[col]}")
    else:
        print(f"Skipping '{col}' due to non-numeric values")
        
# Print ADF test results at the end
for col, result in adf_results.items():
    print(f"ADF Test Result for '{col}': {result}")


### ARIMA Modeling

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from tqdm import tqdm  # Import tqdm

# Create a custom Min-Max scaling function
def custom_min_max_scaling(column):
    min_val = column.min()
    max_val = column.max()
    scaled_column = (column - min_val) / (max_val - min_val)
    return scaled_column

# Scale the numeric columns in the DataFrame (excluding 'date')
numeric_cols = merged_df.columns[1:]
scaled_df = merged_df.copy()
scaled_df[numeric_cols] = scaled_df[numeric_cols].apply(custom_min_max_scaling, axis=0)

# Create subplots with vertical space using Seaborn
plt.figure(figsize=(20, 20))
grid = plt.GridSpec(8, 3, hspace=1.5)  # Adjust the value of hspace as needed

for i, col in enumerate(tqdm(scaled_df.columns[1:], desc="Fitting ARIMA models")):
    ax = plt.subplot(grid[i // 3, i % 3])
    sns.lineplot(data=scaled_df, x="date", y=col, ax=ax)
    ax.set_title(col)
    
    # Rotate x-axis labels to an oblique angle
    plt.xticks(rotation=45)  # You can adjust the angle as needed

    # Create ARIMA model
    model = ARIMA(scaled_df[col], order=(5, 1, 0))
    results = model.fit()

    # Make predictions
    forecast_steps = 24  # Adjust the number of forecasted steps as needed
    forecast = results.forecast(steps=forecast_steps)

    # Plot the original data
    sns.lineplot(data=scaled_df, x="date", y=col, ax=ax, label="Original", linewidth=2)
    
    # Plot the ARIMA model predictions
    forecast_index = pd.date_range(start=scaled_df["date"].max(), periods=forecast_steps, freq="H")
    sns.lineplot(data=pd.DataFrame({"date": forecast_index, col: forecast}), x="date", y=col, ax=ax, label="ARIMA Forecast", linestyle="--")

# Display the plots
plt.show()
